In [1]:
# What is the overlap in the various datasets that may be used for modeling?
import pandas as pd
import numpy as np
from pathlib import Path
import json

#Read Stukalov data
def swap_keys(d):
    d2 = {}
    for key in d:
        t = d[key]
        d2[t] = key
    return d2

def legacy():
    with summary_json_p.open() as f:
        summary_json = json.load(f)
    summary_json = swap_keys(summary_json)
    effectome_p = multi_omnics_p / summary_json[find_key("effectome", summary_json)]

def find_key(query, pydict):
    for key in pydict:
        if query in key:
            return key
    return None
def parse_st_uniprot_ids(ids):
    """
    """
    pid = []
    for i in ids:
        i=i.split(";")
        i=i[0]
        i=i.split("_")
        i=i[0]
        i=i.split('-')[0]
        i=i.split('#')[0]
        pid.append(i)
    return pid

def test_pid(pid):
    errors = []
    for i,j in enumerate(pid):
        try:
            assert len(j) == 6
        except AssertionError:
            errors.append((i,j))
    return errors

def test_upper(s):
    e = []
    for i,v in enumerate(s):
        try:
            assert v.isupper()
        except:
            e.append((i,v))
    return e
            
def read_in_data():
    data_p = Path("../data")
    multi_omnics_p = data_p / "multi-proteomics"
    summary_json_p = multi_omnics_p / "summary.json"
    apms_interactions = multi_omnics_p / '41586_2021_3493_MOESM6_ESM.xlsx'
    st_df = pd.read_excel(apms_interactions, sheet_name=1)
    
    sars_ppi_p = data_p / "sars-cov-2-ppi"
    all_apms = sars_ppi_p / "41586_2020_2286_MOESM5_ESM.xlsx"
    gr_df = pd.read_excel(all_apms, sheet_name=0)
    return st_df, gr_df

def remove_isoforms(df):
    pid = parse_st_uniprot_ids(df['uni'])
    df['uni'] = pid
    return df

def df_preproccessing():
    st_df, gr_df = read_in_data()
    #Rename DataFrames to standard columns
    gr_df.rename(columns={'Bait':'bait', 'PreyGene':'gene', 'Preys':'uni'},inplace=True)
    st_df.rename(columns={'bait_name':'bait','gene_name':'gene',
                          'majority_protein_acs':'uni'},inplace=True)

    #Format uniprot ids to remove isoforms
    gr_df = remove_isoforms(gr_df)
    st_df = remove_isoforms(st_df)
    return gr_df, st_df

gr_df, st_df = df_preproccessing()

def run_tests(df):
    e1 = test_upper(df['uni'])
    e2 = test_upper(df['gene'])
    e3 = test_pid(df['uni'])
    return e1, e2, e3
def run_tests2(d1, d2):
    g1, g2, g3 = run_tests(d1)
    s1, s2, s3 = run_tests(d2)
    assert len(g1)==0
    assert len(s1)==0
    #assert len(g3)==0
    assert len(s3)==0
    print('DF tests passed')
run_tests2(gr_df, st_df)

def intersection(d1, d2, column):
    t = set(d1[column])
    t2 = set(d2[column])
    i = t.intersection(t2)
    return i, len(i)

def test_intersection():
    assert intersection(gr_df, st_df, 'uni') == intersection(st_df, gr_df, 'uni')
test_intersection()

def make_intersection_table(d1, d2):
    i, l =  intersection(d1, d2, 'uni')
    j, l2 = intersection(d1, d2, 'gene')
    b, bl = intersection(d1, d2, 'bait')
    d = pd.DataFrame()
    d.loc['intersection','gene'] = l2
    d.loc['intersection', 'uni'] = l
    d.loc['intersection', 'bait'] = bl
    return d

def get_set_lengths(df):
    d = {}
    for col in df:
        d[col] = len(set(df[col]))
    return d

def define_common_bait_names():
    """
    The actual baits are likely different between the two studies
    """
    


def append_percentages(d):
    gr_set_lengths = get_set_lengths(gr_df)
    st_set_lengths = get_set_lengths(st_df)
    d.loc['gr', ['gene', 'uni','bait']] = [
        gr_set_lengths['gene'], 
        gr_set_lengths['uni'],
        gr_set_lengths['bait']
        ]
    d.loc['st', ['gene', 'uni', 'bait']] = [
        st_set_lengths['gene'],
        st_set_lengths['uni'],
        st_set_lengths['bait']
        ]

intersection_df = make_intersection_table(gr_df, st_df)
append_percentages(intersection_df)
st_bait_names = st_df.iloc[:, 1].unique()
st_gene_names = st_df.iloc[:, 2].unique()
st_uni = st_df.iloc[:, 4].unique()

gr_bait_names = gr_df.iloc[:, 0].unique()
gr_prey_genes = gr_df.iloc[:, 2].unique()
gr_uni = gr_df.iloc[:, 1].unique()

/Users/ajipalar/anaconda3/envs/pynet/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/ajipalar/anaconda3/envs/pynet/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


DF tests passed


In [35]:
AM = np.array( [[0, 1, 0, 0],
                [1, 0, 1, 1],
                [0, 1, 0, 1],
                [0, 1, 1, 0],
                ])

In [40]:
def adj(s, am):
    return np.nonzero(am[s])[0]

In [37]:
remaining = np.random.rand(2, 10)
remaining[0, 0:10] = np.arange(0, 10)
remaining[0, 3] = 9
remaining[0, 9] = 3
b = remaining.copy()

In [44]:
def symmetric_add(s, t, AM):
    AM[s, t] = 1
    AM[t, s] = 1
    return AM
def symmetric_subtract(s, t, AM):
    AM[s, t] = 0
    AM[t, s] = 0
    return AM

In [45]:
a2 = symmetric_subtract(1, 3, AM)

In [48]:
AM == a2

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [2]:
# Output DataFrame
columns = ['baits', 'n prey genes', 'significant_only']
rowname = ['Gordon', 'Stukalov']
dout = pd.DataFrame(columns=columns, index=rowname)

dout.loc[:,'baits'] = [len(gr_bait_names), len(st_bait_names)]
dout.loc[:, 'n prey genes'] = [len(gr_prey_genes), len(st_gene_names)]
dout.loc[:, 'significant_only'] = [False, True]

In [43]:
?np.sort
# quicksort, mergsort, heapsort, stable